<a href="https://colab.research.google.com/github/rebecabrandao1/Transformers-detection/blob/main/CConvNeXt_22K_pred_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import json
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torchvision
import torchvision.transforms as T
from google.colab import drive
drive.mount('/content/drive/')

# check whether run in Colab
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install timm==0.5.4

from timm import create_model

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Running in Colab.


In [3]:
model_name = "convnext_xlarge_in22k"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device = ", device)
# create a ConvNeXt model : https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/convnext.py
model = create_model(model_name, pretrained=True).to(device)

device =  cpu


In [5]:
import os
class LungNoduleDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Transformações
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# DataLoader
image_dir = '/content/drive/MyDrive/my_LUNA/luna_images'
dataset = LungNoduleDataset(image_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Modelo
model = create_model('convnext_xlarge_in22k', pretrained=True)
model.eval()  # Coloque o modelo em modo de avaliação

# Processamento de Imagens em Lote
for images in data_loader:
    images = images.to('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        outputs = model(images)
    #print(outputs)


KeyboardInterrupt: 

In [5]:
def iou(box1, box2):
    """Calcula a interseção sobre união de duas caixas delimitadoras."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Coordenadas da interseção
    x_inter1 = max(x1, x2)
    y_inter1 = max(y1, y2)
    x_inter2 = min(x1 + w1, x2 + w2)
    y_inter2 = min(y1 + h1, y2 + h2)

    if x_inter1 < x_inter2 and y_inter1 < y_inter2:
        inter_area = (x_inter2 - x_inter1) * (y_inter2 - y_inter1)
        box1_area = w1 * h1
        box2_area = w2 * h2
        union_area = box1_area + box2_area - inter_area
        return inter_area / union_area
    else:
        return 0

def evaluate_predictions(true_boxes, pred_boxes):
    """Avalia as previsões com base nos verdadeiros positivos, falsos positivos e recall."""
    TP = 0
    FP = 0
    FN = len(true_boxes)

    for pred_box in pred_boxes:
        matched = False
        for true_box in true_boxes:
            if iou(pred_box, true_box) > 0.5:
                TP += 1
                FN -= 1
                matched = True
                break
        if not matched:
            FP += 1

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0

    return precision, recall


In [6]:
import json


with open('/content/drive/MyDrive/my_LUNA/train.json', 'r') as file:
    data = json.load(file)

annotations = data['annotations']

for annotation in annotations:
    area = annotation['area']
    bbox = annotation['bbox']
    category_id = annotation['category_id']
    annotation_id = annotation['id']
    image_id = annotation['image_id']
    iscrowd = annotation['iscrowd']
    segmentation = annotation['segmentation']

    #print(f"Annotation ID: {annotation_id}, Image ID: {image_id}, Area: {area}, BBox: {bbox}")


In [7]:

def load_true_boxes(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    true_boxes = {}
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        bbox = annotation['bbox']
        if image_id not in true_boxes:
            true_boxes[image_id] = []
        true_boxes[image_id].append(bbox)
    return true_boxes

predicted_boxes = {}


In [8]:
def predict_image(model, image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    image = transform(image).unsqueeze(0)  # Adicionar dimensão de batch
    output = model(image)
    return output

In [2]:

def predict_images_in_directory(model, directory):
    image_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.jpg')]
    predictions = []
    for image_path in image_paths:
        try:
            probabilities = predict_image(model, image_path)
            predictions.append(probabilities)
            #print(f"Predição para imagem {image_path}: {probabilities}")
        except Exception as e:
            print(f"Erro ao prever a imagem {image_path}: {e}")
    return predictions

# Uso exemplo
model.eval()
image_directory = '/content/drive/MyDrive/my_LUNA/luna_images'
all_predictions = predict_images_in_directory(model, image_directory)
#print("Predições para todas as imagens:", all_predictions)
probabilities_tensor = torch.tensor(all_predictions)
pred_class_index = torch.argmax(probabilities_tensor, dim=1)

print("Índice da classe predita:", pred_class_index.item())



NameError: name 'model' is not defined